In [ ]:
import tensorflow as tf
print(tf.__version__)



2.17.1


In [ ]:
# TensorFlow 2.17.1 설치
!pip install tensorflow==2.17.1

# 설치된 TensorFlow 버전 확인
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 455.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 619.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.3/381.3 kB 27.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflo

TensorFlow version: 2.15.0


In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import zipfile
import scipy.stats
from google.colab import drive

# Google Drive mount
drive.mount('/content/drive')

# Define paths
gdrive_zip_file_path = '/content/drive/MyDrive/Pattern_recognition_Assignment2/test_dataset_wo_label.zip'
extract_folder = '/content/drive/MyDrive/Pattern_recognition_Assignment2/test_dataset'
output_file = '/content/drive/MyDrive/Pattern_recognition_Assignment2/result_32204012_HyelimJeon.txt'

# Extract the test dataset zip file
if not os.path.exists(extract_folder):
    with zipfile.ZipFile(gdrive_zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)

# Load the final model
final_model_path = '/content/drive/MyDrive/Pattern_recognition_Assignment2/models3/cnn_model.keras'
model = tf.keras.models.load_model(final_model_path)

# Process test dataset

def load_test_data(data_dir):
    X, filenames = [], []
    for file in sorted(os.listdir(data_dir)):
        if file.endswith(".txt"):
            file_path = os.path.join(data_dir, file)
            data = np.loadtxt(file_path)
            X.append(data)
            filenames.append(file)
    return np.array(X), filenames

def extract_features(X):
    features = []
    for sample in X:
        mean = np.mean(sample, axis=0)
        std = np.std(sample, axis=0)
        minimum = np.min(sample, axis=0)
        maximum = np.max(sample, axis=0)
        median = np.median(sample, axis=0)
        abs_mean = np.mean(np.abs(sample), axis=0)
        skewness = scipy.stats.skew(sample, axis=0)
        kurtosis = scipy.stats.kurtosis(sample, axis=0)
        sma = np.sum(np.abs(sample), axis=0)

        time_features = np.hstack([
            mean, std, minimum, maximum, median, abs_mean, skewness, kurtosis, sma
        ])

        fft = np.fft.rfft(sample, axis=0)
        fft_magnitude = np.abs(fft)
        fft_mean = np.mean(fft_magnitude, axis=0)
        fft_std = np.std(fft_magnitude, axis=0)
        fft_top_n = np.sort(fft_magnitude, axis=0)[-5:].flatten()
        freq_centroid = np.sum(np.arange(fft_magnitude.shape[0])[:, None] * fft_magnitude, axis=0) / np.sum(fft_magnitude, axis=0)
        freq_entropy = -np.sum((fft_magnitude / np.sum(fft_magnitude, axis=0)) * np.log(fft_magnitude / np.sum(fft_magnitude, axis=0) + 1e-10), axis=0)
        power_spectrum = np.sum(fft_magnitude ** 2, axis=0)

        freq_features = np.hstack([
            fft_mean, fft_std, fft_top_n, freq_centroid, freq_entropy, power_spectrum
        ])

        features.append(np.hstack([time_features, freq_features]))
    return np.array(features)

# Load and preprocess test data
data_dir = extract_folder
X_raw, filenames = load_test_data(data_dir)
X_features = extract_features(X_raw)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=min(150, min(X_scaled.shape[0], X_scaled.shape[1])))
X_pca = pca.fit_transform(X_scaled)

X_test = np.expand_dims(X_pca, axis=-1)

# Perform classification
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1) + 1

# Save results
with open(output_file, 'w') as f:
    for filename, prediction in zip(filenames, predicted_classes):
        f.write(f"{filename}: {prediction}\n")

print(f"\nClassification results saved to {output_file}.")


Mounted at /content/drive
93/93 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step

Classification results saved to /content/drive/MyDrive/Pattern_recognition_Assignment2/result_32204012_HyelimJeon.txt.
